# Building Machine Learning Classifiers: Building a basic Random Forest model

### Read in & clean text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()


,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore RandomForestClassifier Attributes & Hyperparameters

In [3]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_n_features', '_estimator_type', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_data', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier()


### Explore RandomForestClassifier through Cross-Validation

In [5]:
from sklearn.model_selection import KFold, cross_val_score

In [7]:
rf= RandomForestClassifier(n_jobs = -1) #parallel processing
k_fold = KFold(n_splits = 5)
cross_val_score(rf, X_features, data['label'], cv = k_fold, scoring = 'accuracy', n_jobs =-1)

array([0.97576302, 0.98114901, 0.97663971, 0.96585804, 0.97304582])

### Through Holdout set

In [8]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size = 0.2)

In [10]:
RF = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs = -1)
rf_model = RF.fit(X_train, y_train)

In [11]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:20]

[(0.07855674383594412, 'body_len'),
 (0.04327310048328036, 3134),
 (0.040804948763059595, 1803),
 (0.03415555721115409, 4796),
 (0.031042504984713584, 7350),
 (0.020200221308452115, 6746),
 (0.019583280229270517, 2031),
 (0.019231994476125832, 5724),
 (0.016886749903822492, 3443),
 (0.01634275708971314, 5988),
 (0.01354526059852918, 392),
 (0.012951555467585332, 7461),
 (0.012088588187276792, 2299),
 (0.011266336920209855, 2171),
 (0.011070244266911045, 5287),
 (0.010770100057368915, 7782),
 (0.010405071366101725, 397),
 (0.010038410416632731, 4295),
 (0.009727014535566854, 7027),
 (0.00891876522334352, 7218)]

In [13]:
y_pred = rf_model.predict(X_test)
precision,recall,fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [25]:
print("precision:{} / recall:{} / Accuracy:{} ".format(precision,recall,(y_pred == y_test).sum()/len(y_pred)))

precision:1.0 / recall:0.6625 / Accuracy:0.9515260323159784 


### Grid-Search

In [29]:
def train_RF(n_est, depth):
    rf_ = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs =-1)
    rf_model_ = rf_.fit(X_train, y_train)
    y_pred_ = rf_model_.predict(X_test)
    precision_, recall_, fscore_, support_ = score(y_test, y_pred, pos_label ='spam', average = 'binary')
    print("Est:{} / Depth:{}------Precision: {} / Recall: {} / Accuracy: {}".format(n_est,depth, precision_, recall_, (y_pred_ == y_test).sum()/len(y_pred)))
    

In [30]:
for n_est in [10, 30, 100]:
    for depth in [10,20,30,None]:
        train_RF(n_est, depth)

Est:10 / Depth:10------Precision: 1.0 / Recall: 0.6625 / Accuracy: 0.8940754039497307
Est:10 / Depth:20------Precision: 1.0 / Recall: 0.6625 / Accuracy: 0.940754039497307
Est:10 / Depth:30------Precision: 1.0 / Recall: 0.6625 / Accuracy: 0.9631956912028725
Est:10 / Depth:None------Precision: 1.0 / Recall: 0.6625 / Accuracy: 0.9703770197486535
Est:30 / Depth:10------Precision: 1.0 / Recall: 0.6625 / Accuracy: 0.895870736086176
Est:30 / Depth:20------Precision: 1.0 / Recall: 0.6625 / Accuracy: 0.9515260323159784
Est:30 / Depth:30------Precision: 1.0 / Recall: 0.6625 / Accuracy: 0.9676840215439856
Est:30 / Depth:None------Precision: 1.0 / Recall: 0.6625 / Accuracy: 0.9757630161579892
Est:100 / Depth:10------Precision: 1.0 / Recall: 0.6625 / Accuracy: 0.8922800718132855
Est:100 / Depth:20------Precision: 1.0 / Recall: 0.6625 / Accuracy: 0.9497307001795332
Est:100 / Depth:30------Precision: 1.0 / Recall: 0.6625 / Accuracy: 0.9658886894075404
Est:100 / Depth:None------Precision: 1.0 / Recall